Pablo Alejandro Aguirre Higuera

Matrícula: 263845

**Práctica #6. Manejo de librería Pandas**

22/08/2025

In [110]:
import pandas as pd
import numpy as np

archivo = "titanic.csv"

**NOTA:** Si se gusta revisar el archivo en Google Colab, favor de importar el Drive y copiar la dirección del csv en la variable archivo. Si se usa Visual Studio Code, hay que asegurarse de tener este ipynb y el csv en la misma carpeta

In [111]:
df = pd.read_csv(archivo)
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


#### **1. Análisis de distribución de supervivencia**

In [112]:
no_survivors = df[df['Survived'] == 0]

no_survivors_grp = no_survivors.groupby(["Pclass", "Sex"]).size()

no_survivors_grp = pd.DataFrame(no_survivors_grp)

print(no_survivors_grp)

                 0
Pclass Sex        
1      female    3
       male     77
2      female    6
       male     91
3      female   72
       male    296


In [113]:
no_survivors_grp = no_survivors_grp.rename(columns={0: 'No Surv Count'})

print(no_survivors_grp)

               No Surv Count
Pclass Sex                  
1      female              3
       male               77
2      female              6
       male               91
3      female             72
       male              296


In [114]:
survivors = df[df['Survived'] == 1]

survivors_grp = survivors.groupby(["Pclass", "Sex"]).size()

survivors_grp = pd.DataFrame(survivors_grp)

print(survivors_grp)

                0
Pclass Sex       
1      female  91
       male    45
2      female  70
       male    17
3      female  72
       male    47


In [115]:
survivors_grp = survivors_grp.rename(columns={0: 'Surv Count'})

print(survivors_grp)

               Surv Count
Pclass Sex               
1      female          91
       male            45
2      female          70
       male            17
3      female          72
       male            47


Ahora se pueden juntar ambos DataFrames para calcular la tasa de sobrevivencia para cada combinación

In [116]:
surv_rates = pd.merge(survivors_grp, no_survivors_grp, on=["Pclass", "Sex"])
surv_rates["Surv Rate"] = (surv_rates["Surv Count"] / (surv_rates["Surv Count"] + surv_rates["No Surv Count"])).round(4)

surv_rates.sort_values(by="Surv Rate", ascending=False, inplace=True) #inplace=True porque están agrupados los datos
surv_rates

,,Surv Count,No Surv Count,Surv Rate
Pclass,Sex,,,
1,female,91,3,0.9681
2,female,70,6,0.9211
3,female,72,72,0.5000
1,male,45,77,0.3689
2,male,17,91,0.1574
3,male,47,296,0.1370


Se ve que la combinación con mejor de tasa de supervivencia es la de mujeres en clase 1 (96.8%), mientras que la combinación de hombres en la clase 3 tuvo la tasa más baja de supervivencia (13.7%).

#### **2. Identificación de familias grandes a bordo**

In [117]:
df['FamilySize'] = df['Siblings/Spouses Aboard'] + df['Parents/Children Aboard']
fam_grandes = df[df['FamilySize'] > 3]

fam_grandes

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,FamilySize
7,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750,4
13,0,3,Mr. Anders Johan Andersson,male,39.0,1,5,31.2750,6
16,0,3,Master. Eugene Rice,male,2.0,4,1,29.1250,5
24,0,3,Miss. Torborg Danira Palsson,female,8.0,3,1,21.0750,4
25,1,3,Mrs. Carl Oscar (Selma Augusta Emilia Johansso...,female,38.0,1,5,31.3875,6
...,...,...,...,...,...,...,...,...,...
820,0,3,Master. Urho Abraham Panula,male,2.0,4,1,39.6875,5
842,0,3,Mr. Douglas Bullen Sage,male,18.0,8,2,69.5500,10
846,0,3,Master. Sigvard Harald Elias Andersson,male,4.0,4,2,31.2750,6
859,0,3,Miss. Dorothy Edith Sage,female,14.0,8,2,69.5500,10


Al imprimir la tabla de pasajeros en familias grandes, se tienen 62 pasajeros en familias grandes

In [118]:
fam_grandes_surv_rate = (fam_grandes["Survived"].sum() / 62).round(4) #Solo suma unos y 62 es el total de pasajeros en familias grandes
print(fam_grandes_surv_rate)

0.1613


La proporción de sobrevivientes en familias grandes es de 16.1%

#### **3. Segmentación por grupos de edad**

In [119]:
#Se pone 19 y no 18 para que indicar que la gente en el rango [0,17] es menor de edad
df["Grupo de Edad"] = pd.cut(df["Age"], bins=[0, 17, np.inf], labels=["Menor de edad", "Mayor de edad"])
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,FamilySize,Grupo de Edad
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500,1,Mayor de edad
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833,1,Mayor de edad
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250,0,Mayor de edad
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000,1,Mayor de edad
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500,0,Mayor de edad


#### **4. Comparación entre promedios calculados manualmente y con Pandas**

In [120]:
age_numpy = np.mean(df["Age"])
fare_numpy = np.mean(df["Fare"])

print(round(age_numpy, 4))
print(round(fare_numpy, 4))

29.4714
32.3054


In [121]:
age_pandas = df["Age"].mean()
fare_pandas = df["Fare"].mean()

print(round(age_pandas, 4))
print(round(fare_pandas, 4))

"Ambas formas dan el mismo resultado" if age_numpy == age_pandas and fare_numpy == fare_pandas else "Los resultados son diferentes"

29.4714
32.3054


'Ambas formas dan el mismo resultado'

#### **5. Creación de intervalos de clase usando NumPy y análisis con Pandas**

In [ ]:
intervalos_fare = np.linspace(df["Fare"].min(), df["Fare"].max(), 5)

df["Fare Interval"] = pd.cut(df["Fare"], bins=intervalos_fare, include_lowest=True) #Include lowest hace que se incluya el 0
df



,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,FamilySize,Grupo de Edad,Fare Interval
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500,1,Mayor de edad,"(-0.001, 128.082]"
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833,1,Mayor de edad,"(-0.001, 128.082]"
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250,0,Mayor de edad,"(-0.001, 128.082]"
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000,1,Mayor de edad,"(-0.001, 128.082]"
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500,0,Mayor de edad,"(-0.001, 128.082]"
...,...,...,...,...,...,...,...,...,...,...,...
882,0,2,Rev. Juozas Montvila,male,27.0,0,0,13.0000,0,Mayor de edad,"(-0.001, 128.082]"
883,1,1,Miss. Margaret Edith Graham,female,19.0,0,0,30.0000,0,Mayor de edad,"(-0.001, 128.082]"
884,0,3,Miss. Catherine Helen Johnston,female,7.0,1,2,23.4500,3,Menor de edad,"(-0.001, 128.082]"
885,1,1,Mr. Karl Howell Behr,male,26.0,0,0,30.0000,0,Mayor de edad,"(-0.001, 128.082]"


In [125]:
df["Fare Interval"].value_counts().sort_index()

Fare Interval
(-0.001, 128.082]     849
(128.082, 256.165]     29
(256.165, 384.247]      6
(384.247, 512.329]      3
Name: count, dtype: int64

La gran mayoría de los pasajeros pagaron una tarifa entre 0 y 128.082 dólares

In [ ]:
#Dado que solo hay unos y ceros en la columna "Survived", la tasa supervivencia se puede calcular con el promedio
fare_interval_surv_rate = df.groupby("Fare Interval")["Survived"].mean().round(4)

fare_interval_surv_rate

C:\Users\pablo\AppData\Local\Temp\ipykernel_19164\3561510600.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fare_interval_surv_rate = df.groupby("Fare Interval")["Survived"].mean().round(4)


Fare Interval
(-0.001, 128.082]     0.3698
(128.082, 256.165]    0.7241
(256.165, 384.247]    0.6667
(384.247, 512.329]    1.0000
Name: Survived, dtype: float64